# Esercizio 2: Automatic Summarization

- L'obiettivo del text summarization è quello di produrre una versione ridotta di un testo che contenga le informazioni importanti o pertinenti.
    - un riassunto di un articolo scientifico, un riepilogo dei thread di posta elettronica, un titolo per un articolo di notizie o i brevi frammenti restituiti dai motori di ricerca web per descrivere ogni documento recuperato.

In [1]:
import re
import math
import spacy 
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd 

nlp = spacy.load('en_core_web_sm',  disable=['ner'])
stigma_words_pos = ["NOUN", "PRON"]
bonus_words_pos = ["ADJ", "ADV"]

stigma_words_list = ['hardly', 'impossible']

##### 1. Caricamento della risorsa NASARI

L'obiettivo è quello di generare un dizionario a partire dal file di testo presente nella cartella NASARI. 
Ogni vettore verrà convertito in una entrata del dizionario con la seguente forma:  

```
{'million': ['bn:00000013n',
  'million',
  ...,
  'infinity'],
 'day': ['bn:00000086n',
  'day',
  ...,
  'time',
  'clock']
}
```

In [2]:
def initialize_ddsmallnasari():
    ddsmallnasari = {}

    with open(f'NASARI/dd-small-nasari-15.txt', 'r', encoding='utf8') as f:
        lines = f.readlines()

    for line in lines:
        parts = line.split(";")
        key = parts[1].lower()
        var = []
        for p in parts:
            u = p.split("_")
            var.append(u[0].lower())
        if key not in ddsmallnasari:
            ddsmallnasari[key] = [var]
        else: 
            ddsmallnasari[key].append(var)
    
    return ddsmallnasari

In [3]:
ddsmallnasari = initialize_ddsmallnasari()
ddsmallnasari

{'million': [['bn:00000013n',
   'million',
   'million',
   'number',
   'mathematics',
   'long scale',
   'real number',
   'numeral',
   'short scale',
   'digit',
   'bally',
   'millionaire',
   'penguin',
   'markov',
   'complex number',
   'infinity']],
 'day': [['bn:00000086n',
   'day',
   'day',
   'planet',
   'sun',
   'earth',
   'calendar',
   'time',
   'clock',
   'moon',
   'orbit',
   'month',
   'hour',
   'decimal',
   'astronomical',
   'leap year']],
 'quarto': [['bn:00000116n',
   'quarto',
   'shakespeare',
   'quarto',
   'play',
   'henry vi',
   'edition',
   'hamlet',
   'folio',
   'titus',
   'shrew',
   'print',
   'sonnet',
   'octavo',
   'petruchio',
   'henry']],
 'serotonin': [['bn:00000122n',
   'serotonin',
   'serotonin',
   '5-ht',
   'receptor',
   'agonist',
   '2c',
   'neurotransmitter',
   'leptin',
   'drug',
   'tryptophan',
   'dopamine',
   'serotonergic',
   'antagonist',
   'antidepressant',
   'neuron']],
 'sextet': [['bn:00000125n'

##### 2. Caricamento dei documenti da riassumere

In [4]:
def read_doc(doc: str):
    document = []
    
    with open('docs/{}'.format(doc), 'r', encoding="utf8") as f:
        lines = f.readlines()
        for line in lines:
            if line != "\n":
                cleanString = re.sub(r"[^a-zA-Z0-9]+", ' ', line)
                document.append(cleanString)

    if doc != "Trump-wall.txt":
        title = document[0]
        document = document[1:] # remove title from document
    else: # if there is no title, use the filename cleaned as title
        title = "Trump wall" 

    return document, title.rstrip()

In [5]:
files = ["Andy-Warhol.txt", "Ebola-virus-disease.txt", "Life-indoors.txt", "Napoleon-wiki.txt", "Trump-wall.txt"]

docs = []
for file in files: 
    document, title = read_doc(file)
    docs.append((title, document))
    print("Title: ", title)
    print()

Title:  Andy Warhol Why the great Pop artist thought Trump is sort of cheap

Title:  Ebola virus disease

Title:  How people around the world are coping with life indoors

Title:  Napoleone Bonaparte

Title:  Trump wall



##### 3. Creazione del contesto del documento

Per individuare le parole e porzioni di testo rilevanti in termini di contenuto, sono stati implementati due **criteri di rilevanza** differenti: 
- *Metodo del titolo*: l'idea è che il titolo abbia informazioni rilevanti riguardo il contenuto del testo, per questo vengono individuati i vettori nasari a partire dalle parole presenti;
- *Cue phrases method*: in cui vengono individuate le frasi che contengono parole che ci permettono di capire che stanno per essere dette cose importanti (bonus phrases) o inutili (stigma phrases). Le frasi più importanti in questo modo vengono individuate e vengono estratti i corrispettivi vettori nasari delle parole che contengono. 
- *Metodo ibrido*: il contesto viene generato utilizzando entrambe le metodologie precedenti, quindi vengono utilizzate sia le informazioni delle frasi più importanti che il titolo del documento.

In [6]:
def bag_of_words(sentence: str) -> tuple[list, list]: # Preprocessing

    tokens = nlp(sentence)
    tokens = [token for token in tokens if token.is_alpha and not token.is_stop]
    tokens_text = [token.lemma_ for token in tokens]

    return tokens_text, tokens

In [7]:
def get_context(words: list) -> list: 
    """
    Questo metodo prende in input una lista di token e ritorna una lista di vettori nasari 
    che rappresentano il contesto del documento.
    """
    context = [] 

    for word in words:
        try:
            context.extend(ddsmallnasari[word.lower()])
        except:
            pass

    return context

def get_cue_score(words: list) -> int:
    """
    Questo metodo prende in input una lista di token che identificano una frase nel testo e
    restituisce un punteggio che indica l'importanza della frase. 
    Il punteggio si basa sulla presenza di parole che ci permettono di capire 
    che stanno per essere dette cose importanti e quindi il valore della frase al fine 
    della generazione del riassunto del documento a cui appartiene. 
    """

    cue_score = 0
    for word in words:
        if word.pos_ in stigma_words_pos:
            cue_score -= 1
        elif word.pos_ in bonus_words_pos:
            if word.text.lower() in stigma_words_list:
                cue_score -= 1
            else:
                cue_score += 1
    
    return cue_score



In [8]:
threshold = 10 # numero minimo di vettori nasari per costruire un contesto

def get_context_title_method(title: str) -> list: 
    """ 
    Questo metodo prende in input il titolo del documento e ritorna una lista di vettori nasari
    che rappresentano il contesto del titolo.
    
    Nel caso in cui il titolo sia troppo breve e/o quindi poco informativo, il metodo cerca di
    individuare altri vettori nasari che contengano almeno una parola del titolo.

    """

    tokens, _ = bag_of_words(title)
    context = get_context(tokens)

    if len(context) < threshold:  # se il titolo è troppo breve e/o quindi poco informativo
        for elem in ddsmallnasari:
            if len(context) >= threshold:
                break
            vectors = ddsmallnasari[elem].copy()
            for vector in vectors:
                for el in vector:
                    if el in tokens:
                        context.append(vector)
                        break  # passo al prossimo vettore nasari

    return context


def get_context_cue_method(doc: list) -> list: 
    """
    Questo metodo prende in input il documento e restituisce una lista di vettori nasari. 
    Il metodo individua la frase più importante del documento e ne estrae il contesto.
    """

    scores = [(sentence, get_cue_score(bag_of_words(sentence)[1])) for sentence in doc]
   
    scores.sort(key=lambda x: x[1], reverse=True)

    context = []
    for sentence, _ in scores:
        if len(context) >= threshold:
            break
        context_bow, _ = bag_of_words(sentence)
        context.extend(get_context(context_bow))
    
    return context


def get_context_hybrid(title: str, doc: list) -> list:
    """
    Questo metodo prende in input il titolo del documento e il documento stesso e restituisce
    una lista di vettori nasari che rappresentano il contesto del documento.
    """

    context = get_context_title_method(title)
    context.extend(get_context_cue_method(doc))
    
    return context


In [9]:
contexts_title = []
contexts_cue = []
contexts_hybrid = []

for title, document in docs:
    contexts_title.append((title, document, get_context_title_method(title)))
    contexts_cue.append((title, document, get_context_cue_method(document)))
    contexts_hybrid.append((title, document, get_context_hybrid(title, document)))

In [10]:
print("TITLE METHOD")
for title, document, context in contexts_title:
    print(title)
    print("first 3 vectors: ", context[:3])
    print("len: ", len(context))
    print()

print("CUE METHOD")
for title, document, context in contexts_cue:
    print(title)
    print("first 3 vectors: ", context[:3])
    print("len: ", len(context))
    print() 

print("HYBRID METHOD")
for title, document, context in contexts_hybrid:
    print(title)
    print("first 3 vectors: ", context[:3])
    print("len: ", len(context))
    print()

TITLE METHOD
Andy Warhol Why the great Pop artist thought Trump is sort of cheap
first 3 vectors:  [['bn:03250251n', 'great', 'graph', 'transformation', 'ocl', 'rewrite', 'qvt', 'language', 'model', 'meta-object facility', 'sublanguage', 'uml', 'mde', 'mof', 'domain-specific modelling', 'omg'], ['bn:00006182n', 'artist', 'artist', 'duchamp', 'art', 'audubon', 'hirschfeld', 'tiffany', 'chucky', 'work', 'drawing', 'lear', 'ankomah', 'painting', 'landers', 'magalios'], ['bn:00078467n', 'trump', 'card', 'trump', 'trick', 'player', 'bid', 'suit', 'tarot', 'play', 'hand', 'deck', 'point', 'game', 'bidding', 'declarer']]
len:  10

Ebola virus disease
first 3 vectors:  [['bn:00080085n', 'virus', 'virus', 'viral', 'cell', 'genome', 'rna', 'protein', 'dna', 'infect', 'host', 'infection', 'capsid', 'replication', 'gene', 'disease'], ['bn:00027546n', 'disease', 'disease', 'health', 'infection', 'symptom', 'patient', 'cause', 'treatment', 'chronic', 'pathogen', 'kawasaki disease', 'epidemiology', '

##### 4. Calcolo della Weighted Overlap

La WO è definita come la sovrapposizione pesata che si basa sul rango, ossia su quante volte un termine occorre. 



In [11]:
def WO(vector1: list[str], vector2: list[str]) -> float: # Weighted Overlap

    wo = 0
    numeratore = 0
    denominatore = 0
    overlap = set(vector1).intersection(set(vector2)) # insieme di dimensioni sovrapposte

    # calcolo numeratore
    for token in overlap:
        rankv1 = vector1.index(token) + 1 # + 1 per evitare di poter avere 0 al denominatore
        rankv2 = vector2.index(token) + 1 # + 1 per evitare di poter avere 0 al denominatore
        numeratore += 1/(rankv1 + rankv2)
    
    # calcolo denominatore
    for i in range(1, len(overlap) + 1):
        denominatore += 1/(2*i)
        
    if denominatore != 0:
        wo = numeratore/denominatore
    
    return wo
    

##### 5. Calcolo della similarità 
L'obiettivo è quello di calcolare la similarità tra due parole w1 e w2 come la similarità dei due sensi più vicini tra loro

In [12]:
def similarity(word: str, context: list[list[str]]) -> float:
    """
    Calcola la similarità tra una parola e il contesto composto da vettori nasari.
    La parola deve essere presente nel dizionario ddsmallnasari.
    """
    
    values_similarity = [] 
    word_nasari = ddsmallnasari[word.lower()].copy() 

    for vector_nasari in word_nasari: 
        for context_nasari in context:
            val = WO(vector_nasari[2:], context_nasari[2:])
            values_similarity.append(val)

    values_similarity_sqrt = [math.sqrt(val) for val in values_similarity] 

    return max(values_similarity_sqrt) 

##### 6. Pesatura delle frasi del documento

In [13]:
def calculate_weight_of_paragraphs(document: list[str], context: list) -> list[float]:

    weighted_array = []
    for line in document:
        words_line = line.split(" ") # rimuovo l'ultimo elemento che è il carattere di newline

        value = 0

        if len(context) > 0:
            # Per ogni parola nella frase si calcola la similarità con le parole del contesto
            for word in words_line:
                if word in ddsmallnasari:
                    value += similarity(word, context)

            weighted_array.append(value / len(words_line))

        else: 
            # Se il contesto è vuoto, ogni frase avrà peso 1 (non è possibile calcolare la similarità)
            weighted_array.append(1)
    
    return weighted_array


In [14]:
weighted_paragraphs_title = []
for title, document, context in contexts_title:
    weighted_paragraphs_title.append(calculate_weight_of_paragraphs(document, context))

weighted_paragraphs_cue = []
for title, document, context in contexts_cue:
    weighted_paragraphs_cue.append(calculate_weight_of_paragraphs(document, context))

weighted_paragraphs_hybrid = []
for title, document, context in contexts_hybrid:
    weighted_paragraphs_hybrid.append(calculate_weight_of_paragraphs(document, context))

##### 7. Generazione del riassunto

In [15]:
def summarize(index_doc: int, document: list, compress_percentage: int, weighted_paragraphs: list, method: str = 'title') -> list:

    filename = f'summarized_docs/' + str(compress_percentage) + '/'+ files[index_doc].replace('.txt', '') + '_' + method + '.txt'
    
    result = enumerate(weighted_paragraphs)
    new_len_perc = float(100 - compress_percentage)
    size = (int)(len(weighted_paragraphs) * new_len_perc / 100)
    desc_sorted_weighted_paragraps = sorted(result, key=lambda x: x[1], reverse=True)

    desc_sorted_weighted_paragraps = desc_sorted_weighted_paragraps[:size] # primi size elementi che hanno peso maggiore

    asc_sorted_weighted_paragraps = sorted(desc_sorted_weighted_paragraps, key=lambda x: x[0])

    summarization = []
    for phrase_index, score in asc_sorted_weighted_paragraps:
        if score > 0:
            summarization.append(document[phrase_index])

    with open(filename, 'w', encoding='utf8') as f:
        for line in summarization:
            f.write(line)
            f.write('\n')
    
    return summarization

In [16]:
summaries_title = []
summaries_cue = []
summaries_hybrid = []
compress_percentages = [20, 30, 40] 

for compress_percentage in compress_percentages:
    for i in range(0, len(docs)):
        summaries_title.append(summarize(i, docs[i][1], compress_percentage, weighted_paragraphs_title[i], method='title'))
        summaries_cue.append(summarize(i, docs[i][1], compress_percentage, weighted_paragraphs_cue[i], method='cue'))
        summaries_hybrid.append(summarize(i, docs[i][1], compress_percentage, weighted_paragraphs_hybrid[i], method='hybrid'))


In [17]:
for i in range(0, len(files)):
    print(files[i])
    print("Lunghezza riassunto con title method: " + str(len(summaries_title[i])) + " frasi")
    print("Lunghezza riassunto con cue method: " + str(len(summaries_cue[i])) + " frasi")
    print()

Andy-Warhol.txt
Lunghezza riassunto con title method: 16 frasi
Lunghezza riassunto con cue method: 16 frasi

Ebola-virus-disease.txt
Lunghezza riassunto con title method: 19 frasi
Lunghezza riassunto con cue method: 19 frasi

Life-indoors.txt
Lunghezza riassunto con title method: 9 frasi
Lunghezza riassunto con cue method: 9 frasi

Napoleon-wiki.txt
Lunghezza riassunto con title method: 13 frasi
Lunghezza riassunto con cue method: 13 frasi

Trump-wall.txt
Lunghezza riassunto con title method: 40 frasi
Lunghezza riassunto con cue method: 46 frasi



##### 8. Valutazione del sistema di riassunto automatico

In [18]:
def get_relevant_terms(doc: list[str]) -> set[str]:
    """
    Prende in input un documento e restituisce una lista di termini rilevanti
    utilizzando la matrice TF-IDF. 
    """
    vectorizer = TfidfVectorizer(stop_words='english')
    tf_idf = vectorizer.fit_transform(doc)
    df = pd.DataFrame(tf_idf.todense(), columns=vectorizer.get_feature_names_out())
    score_words = df.mean(axis=0)
    score_words = score_words.sort_values(ascending=False)

    perc = (100 - compress_percentage) / 100 # le prime (100 - riduzione)% delle parole con score più alto
    threshold = int(round(len(score_words) * perc))
    score_words = score_words[:threshold]

    important_words = set([item[0] for item in score_words.items()])
    return important_words

def get_blue_evaluation(relevant_terms: set[str], candidate_terms: set[str]) -> float:
    
    intersection = relevant_terms.intersection(candidate_terms)
    return len(intersection) / len(candidate_terms)

def get_rouge_evaluation(relevant_terms: set[str], candidate_terms: set[str]) -> float:
    
    intersection = relevant_terms.intersection(candidate_terms)
    return len(intersection) / len(relevant_terms)

In [19]:
def evaluation(index_doc: int, method: str):
    
    filename = f'docs/' + files[index_doc]
    summarization_file_name = f'summarized_docs/' + str(compress_percentage) + '/' + files[index_doc].replace('.txt', '') + '_' + method + '.txt'
    
    doc = []
    summary = []
    with open(filename, 'r', encoding='utf8') as f:
        doc = f.readlines()

    with open(summarization_file_name, 'r', encoding='utf8') as f:
        summary = f.readlines()

    relevant_terms = get_relevant_terms(doc)
    
    candidate_terms = set()

    for line in summary:
        candidate_terms.update(bag_of_words(line)[0])

    blue = get_blue_evaluation(relevant_terms, candidate_terms)
    rouge = get_rouge_evaluation(relevant_terms, candidate_terms)

    print("PRECISION: " + str(blue * 100) + " %") # BLUE
    print("RECALL: " + str(rouge * 100)+ " %") # ROUGE

    return blue, rouge
    

In [20]:
for i in range(0, len(files)):
    print(files[i])
    print("Evalutation title method:")
    evaluation(i, "title")
    print("Evalutation cue method:")
    evaluation(i, "cue")
    print("Evalutation hybrid method:")
    evaluation(i, "hybrid")
    print("\n\n")

Andy-Warhol.txt
Evalutation title method:
PRECISION: 38.69731800766284 %
RECALL: 34.707903780068726 %
Evalutation cue method:
PRECISION: 38.82783882783883 %
RECALL: 36.42611683848797 %
Evalutation hybrid method:
PRECISION: 37.5 %
RECALL: 34.02061855670103 %



Ebola-virus-disease.txt
Evalutation title method:
PRECISION: 38.13229571984436 %
RECALL: 50.90909090909091 %
Evalutation cue method:
PRECISION: 38.13229571984436 %
RECALL: 50.90909090909091 %
Evalutation hybrid method:
PRECISION: 41.63265306122449 %
RECALL: 52.98701298701298 %



Life-indoors.txt
Evalutation title method:
PRECISION: 41.228070175438596 %
RECALL: 37.6 %
Evalutation cue method:
PRECISION: 40.17857142857143 %
RECALL: 36.0 %
Evalutation hybrid method:
PRECISION: 39.66942148760331 %
RECALL: 38.4 %



Napoleon-wiki.txt
Evalutation title method:
PRECISION: 34.97536945812808 %
RECALL: 31.277533039647576 %
Evalutation cue method:
PRECISION: 37.5 %
RECALL: 34.36123348017621 %
Evalutation hybrid method:
PRECISION: 37.5 %
REC